In [8]:
import os
import json
import polars as pl
from pathlib import Path

## Turn File into .parquet

In [9]:
INPUT_DIR = "data/raw_youtube_comments"
OUTPUT_FILE = "all_comments.parquet"

In [10]:
def flatten_json(d, parent_key="", sep="_"):
    """Recursively flattens a nested JSON object into a dict with prefixed keys."""
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_json(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [ ]:
# # Let's say you have multiple JSON files in a folder
# data_folder = Path("data/raw_youtube_comments_relevance")
# rows = []

# for file in data_folder.glob("*.json"):
#     with open(file, "r", encoding="utf-8") as f:
#         raw = json.load(f)
#         for entry in raw.get("items", []):  # your "response" field
#             rows.append(flatten_json(entry))

# # Create Polars DataFrame
# df = pl.DataFrame(rows)

# # Save to parquet
# df.write_parquet("comments.parquet")

## Data by relevance

In [ ]:
df = pl.read_parquet("data/clean/relevance/comments.parquet")
df.head()

kind,etag,id,snippet_channelId,snippet_videoId,snippet_topLevelComment_kind,snippet_topLevelComment_etag,snippet_topLevelComment_id,snippet_topLevelComment_snippet_channelId,snippet_topLevelComment_snippet_videoId,snippet_topLevelComment_snippet_textDisplay,snippet_topLevelComment_snippet_textOriginal,snippet_topLevelComment_snippet_authorDisplayName,snippet_topLevelComment_snippet_authorProfileImageUrl,snippet_topLevelComment_snippet_authorChannelUrl,snippet_topLevelComment_snippet_authorChannelId_value,snippet_topLevelComment_snippet_canRate,snippet_topLevelComment_snippet_viewerRating,snippet_topLevelComment_snippet_likeCount,snippet_topLevelComment_snippet_publishedAt,snippet_topLevelComment_snippet_updatedAt,snippet_canReply,snippet_totalReplyCount,snippet_isPublic,replies_comments
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool,str,i64,str,str,bool,i64,bool,list[struct[4]]
"""youtube#commentThread""","""5X4c9JpM8u0seA5iAaQ8P9Z30UE""","""Ugx_LrD9EQxPmvmHmNB4AaABAg""","""UCHkj014U2CQ2Nv0UZeYpE_A""","""kffacxfA7G4""","""youtube#comment""","""OFySluPd6L6HxyxATAhAZLZIu88""","""Ugx_LrD9EQxPmvmHmNB4AaABAg""","""UCHkj014U2CQ2Nv0UZeYpE_A""","""kffacxfA7G4""","""Virgo""","""Virgo""","""@BilalRazzaq-r4p""","""https://yt3.ggpht.com/ytc/AIdr…","""http://www.youtube.com/@BilalR…","""UCbyVaZZErCfW8kV0vPUMUqQ""",true,"""none""",0,"""2025-10-02T10:12:32Z""","""2025-10-02T10:12:32Z""",true,0,true,null
"""youtube#commentThread""","""03_v_OV1dSXBxKOHHVoh9ddg5VY""","""UgwL0V8QLNB_bmXMEYx4AaABAg""","""UCHkj014U2CQ2Nv0UZeYpE_A""","""kffacxfA7G4""","""youtube#comment""","""BOyJO_vmuZ5Y20upJQHQVXlNi6Y""","""UgwL0V8QLNB_bmXMEYx4AaABAg""","""UCHkj014U2CQ2Nv0UZeYpE_A""","""kffacxfA7G4""","""Azgona""","""Azgona""","""@lindaselimi1723""","""https://yt3.ggpht.com/ytc/AIdr…","""http://www.youtube.com/@lindas…","""UCBwnn2MNaIW1rWr1J5XMoRg""",true,"""none""",0,"""2025-09-26T12:05:50Z""","""2025-09-26T12:05:50Z""",true,0,true,null
"""youtube#commentThread""","""zijVJD5RGsUJFjt_v-8uXlf2e2E""","""Ugx9WwEYettdsDcZ7MV4AaABAg""","""UCHkj014U2CQ2Nv0UZeYpE_A""","""kffacxfA7G4""","""youtube#comment""","""s8On9I5xCwCWXhDdqBPBl2kELoY""","""Ugx9WwEYettdsDcZ7MV4AaABAg""","""UCHkj014U2CQ2Nv0UZeYpE_A""","""kffacxfA7G4""","""Justin <a href=""https://www.yo…","""Justin 2:31 hoy no puedo tengo…","""@JuliethandreaAsprilla""","""https://yt3.ggpht.com/_gSgbkBK…","""http://www.youtube.com/@Juliet…","""UC7_7zuBcOWGw4NzkWynlbXg""",true,"""none""",0,"""2025-09-26T15:38:02Z""","""2025-09-26T15:38:02Z""",true,0,true,null
"""youtube#commentThread""","""12t3pbgH0bzDzU7UMs7NfLeNhdA""","""UgzWEHA_DyX6lhxXGRt4AaABAg""","""UCHkj014U2CQ2Nv0UZeYpE_A""","""kffacxfA7G4""","""youtube#comment""","""_hw2ZYC0sL_4Bl5YcO5psTFeOEU""","""UgzWEHA_DyX6lhxXGRt4AaABAg""","""UCHkj014U2CQ2Nv0UZeYpE_A""","""kffacxfA7G4""","""Is this before he got diddled?""","""Is this before he got diddled?""","""@Dr3w_4_u""","""https://yt3.ggpht.com/ytc/AIdr…","""http://www.youtube.com/@Dr3w_4…","""UCJ49JtaIp3vCVjKl1QtfSXg""",true,"""none""",0,"""2025-09-29T23:22:35Z""","""2025-09-29T23:22:35Z""",true,0,true,null
"""youtube#commentThread""","""2Bs4BWOg2QNNGKEDsZkWlTAtl6Q""","""UgzeBP3fOh430MO3Ohx4AaABAg""","""UCHkj014U2CQ2Nv0UZeYpE_A""","""kffacxfA7G4""","""youtube#comment""","""uSqWR9fQEzeZkvrq9Ulqyd_I4fU""","""UgzeBP3fOh430MO3Ohx4AaABAg""","""UCHkj014U2CQ2Nv0UZeYpE_A""","""kffacxfA7G4""","""ロジック😮""","""ロジック😮""","""@user-zd8wp8bl3v""","""https://yt3.ggpht.com/ytc/AIdr…","""http://www.youtube.com/@user-z…","""UC6vJc-nIXA7NXzQbIftId_g""",true,"""none""",0,"""2025-09-27T19:18:18Z""","""2025-09-27T19:18:18Z""",true,0,true,null


In [13]:
df.shape

(1086, 25)

## Data by Chronological Order

In [14]:
df = pl.read_parquet("data/clean/chronological/comments.parquet")
df.head()

kind,etag,id,snippet_channelId,snippet_videoId,snippet_topLevelComment_kind,snippet_topLevelComment_etag,snippet_topLevelComment_id,snippet_topLevelComment_snippet_channelId,snippet_topLevelComment_snippet_videoId,snippet_topLevelComment_snippet_textDisplay,snippet_topLevelComment_snippet_textOriginal,snippet_topLevelComment_snippet_authorDisplayName,snippet_topLevelComment_snippet_authorProfileImageUrl,snippet_topLevelComment_snippet_authorChannelUrl,snippet_topLevelComment_snippet_authorChannelId_value,snippet_topLevelComment_snippet_canRate,snippet_topLevelComment_snippet_viewerRating,snippet_topLevelComment_snippet_likeCount,snippet_topLevelComment_snippet_publishedAt,snippet_topLevelComment_snippet_updatedAt,snippet_canReply,snippet_totalReplyCount,snippet_isPublic,replies_comments
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool,str,i64,str,str,bool,i64,bool,list[struct[4]]
"""youtube#commentThread""","""uQA-qS9CXMbmpJnBFzzYVKNAsvM""","""UgxJOtvRgP-TK-DDSbh4AaABAg""","""UCHkj014U2CQ2Nv0UZeYpE_A""","""kffacxfA7G4""","""youtube#comment""","""ftCX1u_cMcFYWvy1sForf9HYYlo""","""UgxJOtvRgP-TK-DDSbh4AaABAg""","""UCHkj014U2CQ2Nv0UZeYpE_A""","""kffacxfA7G4""","""Good reason holy grain""","""Good reason holy grain""","""@nishapatel-m8r""","""https://yt3.ggpht.com/ytc/AIdr…","""http://www.youtube.com/@nishap…","""UC1pcc2njrSbt9Dxvv7jCZcQ""",true,"""none""",1,"""2025-09-24T17:11:11Z""","""2025-09-24T17:11:11Z""",true,0,true,null
"""youtube#commentThread""","""4hudTL3Z5iN4pgKkkQbIulOJ5EM""","""UgxtC2RpNWCNRupMHvl4AaABAg""","""UCHkj014U2CQ2Nv0UZeYpE_A""","""kffacxfA7G4""","""youtube#comment""","""rTwa0LUA1izDUGc0PKXTIu-GiJg""","""UgxtC2RpNWCNRupMHvl4AaABAg""","""UCHkj014U2CQ2Nv0UZeYpE_A""","""kffacxfA7G4""","""anyone in 2025 ....... lets ga…","""anyone in 2025 ....... lets ga…","""@moloantoa8655""","""https://yt3.ggpht.com/sCWKiUE8…","""http://www.youtube.com/@moloan…","""UCHLFXF2C8LdrjPhH3Vi4dyA""",true,"""none""",56,"""2025-09-24T16:47:44Z""","""2025-09-24T16:47:44Z""",true,8,true,"[{""youtube#comment"",""aaOFfL5gP8hlrICa-lZ1PtGajTc"",""UgxtC2RpNWCNRupMHvl4AaABAg.ANT03nbSg30ANTQ5hnoIcR"",{""UCHkj014U2CQ2Nv0UZeYpE_A"",""kffacxfA7G4"",""hehe"",""hehe"",""UgxtC2RpNWCNRupMHvl4AaABAg"",""@Toliii_"",""https://yt3.ggpht.com/zOobxWViS3ZZQ4T5dmDK3bkleUQZGHV4sPgxanVnhUeqtcmYMBVXtIlzRVNRR_3Sqqu-Px0IRc4=s48-c-k-c0x00ffffff-no-rj"",""http://www.youtube.com/@Toliii_"",{""UCmiE9AN4VP-mbZLe4YZIUeA""},true,""none"",2,""2025-09-24T20:35:12Z"",""2025-09-24T20:35:12Z""}}, {""youtube#comment"",""RApwV-DBVk3KyXkgj5WdBT7wVg8"",""UgxtC2RpNWCNRupMHvl4AaABAg.ANT03nbSg30ANTc5AYS6rZ"",{""UCHkj014U2CQ2Nv0UZeYpE_A"",""kffacxfA7G4"",""Here bro, just for nostalgia"",""Here bro, just for nostalgia"",""UgxtC2RpNWCNRupMHvl4AaABAg"",""@AssSlayer-v4q"",""https://yt3.ggpht.com/fJvVUbGK5uNPv-jG7cX4v0N_-VNNUFzOSqeMOBPuAgqtqY30T9Pj2DaYq3-tCJIsemU5_BeC-w=s48-c-k-c0x00ffffff-no-rj"",""http://www.youtube.com/@AssSlayer-v4q"",{""UCBELkmM6LYQ02rNyibcTrTA""},true,""none"",0,""2025-09-24T22:28:43Z"",""2025-09-24T22:28:43Z""}}, … {""youtube#comment"",""z2aiV_-BciedbhIBylMPA4XAMaY"",""UgxtC2RpNWCNRupMHvl4AaABAg.ANT03nbSg30ANWq7skb7fa"",{""UCHkj014U2CQ2Nv0UZeYpE_A"",""kffacxfA7G4"",""Hey"",""Hey"",""UgxtC2RpNWCNRupMHvl4AaABAg"",""@Eliotxpresso"",""https://yt3.ggpht.com/cXSFxB5g6oLy2imLdKik8qZ6UpUFVKiXus1JCoE8wxja8tWVEl0Q6Dn6qUTSd9cruBPYBS0U=s48-c-k-c0x00ffffff-no-rj"",""http://www.youtube.com/@Eliotxpresso"",{""UC3v-Xu6dpdHX0kZva3NnJXA""},true,""none"",1,""2025-09-26T04:29:08Z"",""2025-09-26T04:29:08Z""}}]"
"""youtube#commentThread""","""Mh2HvwFyqTHuY3lmXdtWdqTXbSs""","""UgxKAugIlYmsTGIUgxt4AaABAg""","""UCHkj014U2CQ2Nv0UZeYpE_A""","""kffacxfA7G4""","""youtube#comment""","""VUb_AWjx-fRjqJH9VA1DiG7tWhc""","""UgxKAugIlYmsTGIUgxt4AaABAg""","""UCHkj014U2CQ2Nv0UZeYpE_A""","""kffacxfA7G4""","""After so many years people r s…","""After so many years people r s…","""@Meckzy_on60fps""","""https://yt3.ggpht.com/7OthEhNG…","""http://www.youtube.com/@Meckzy…","""UCfdxsCfK0qJG

In [15]:
df.shape

(2000, 25)